In [29]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Define the data directory
data_dir = '/data'
#os.chdir(data_dir)
print(os.getcwd())  # Verify the current working directory


/home/jovyan/FunGut2/John


Checking out the reads

In [ ]:
! qiime demux summarize \
  --i-data ./data/fungut_forward_reads.qza \
  --o-visualization ./data/fungut_forward_reads_summary.qzv

In [ ]:
Visualization.load('/home/jovyan/FunGut2/John/data/fungut_forward_reads_summary.qzv')

Checking out the metadata

In [ ]:
# Check metadata file formatting
!qiime metadata tabulate \
  --m-input-file /home/jovyan/FunGut2/John/data/fungut_metadata.tsv \
  --o-visualization /home/jovyan/FunGut2/John/data/metadata.qzv

In [9]:
Visualization.load('/home/jovyan/FunGut2/John/data/metadata.qzv')

<visualization: Visualization uuid: 94e9adb0-0387-42c8-85a0-9d502ffdb659>

Denoising step

In [7]:
!qiime tools peek ./data/rep-seqs.qza

UUID:        1866201b-f5b3-4750-9a79-c22d6031a2aa
Type:        FeatureData[Sequence]
Data format: DNASequencesDirectoryFormat


In [ ]:
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs ./data/fungut_forward_reads.qza \
  --p-trim-left 0 \
  --p-trunc-len 150 \
  --o-representative-sequences ./data/rep-seqs.qza \
  --o-table ./data/table.qza \
  --o-denoising-stats ./data/denoising-stats.qza

Classification

In [ ]:
'''
!qiime feature-classifier classify-sklearn \
  --i-classifier ./data/classifiers/'unite_ver10_99_04.04.2024-Q2-2024.5.qza' \
  --i-reads ./data/rep-seqs.qza \
  --p-reads-per-batch 10 \
  --p-n-jobs 0 \
  --o-classification ./taxonomy.qza
'''

In [ ]:
# tabulation of the taxonomy classification only 99, so the too basic classifier, without eukaryotes
!qiime metadata tabulate \
  --m-input-file ./data/taxonomy.qza \
  --o-visualization ./data/taxonomy.qzv

In [ ]:
Visualization.load('/home/jovyan/FunGut2/John/data/taxonomy.qzv')

In [ ]:
!qiime metadata tabulate \
  --m-input-file ./taxonomy_unite_dynamic_s_all.qza \
  --o-visualization ./taxonomy_unite_dynamic_s_all.qzv

In [15]:
Visualization.load('./taxonomy_unite_dynamic_s_all.qzv')

<visualization: Visualization uuid: 5ae77c4a-c5b8-46c0-8b57-5f6234aa07ba>

In [ ]:
!qiime taxa barplot \
  --i-table ./data/table.qza \
  --i-taxonomy ./taxonomy_unite_dynamic_s_all.qza \
  --m-metadata-file ./data/fungut_metadata.tsv \
  --o-visualization ./data/taxa-bar-plots.qzv


In [ ]:
Visualization.load('./data/taxa-bar-plots.qzv')

In [12]:
!qiime feature-table summarize \
  --i-table ./data/table.qza \
  --o-visualization ./data/table-summary.qzv

Saved Visualization to: ./data/table-summary.qzv


In [ ]:
Visualization.load('./data/table-summary.qzv')

Filtering the feature table to exclude eukaryotes

In [17]:
!qiime taxa filter-table \
  --i-table ./data/table.qza \
  --i-taxonomy ./taxonomy_unite_dynamic_s_all.qza \
  --p-exclude Eukaryota,Unassigned \
  --o-filtered-table ./data/filtered-feature-table.qza


Saved FeatureTable[Frequency] to: ./data/filtered-feature-table.qza


In [21]:
!qiime feature-table summarize\
    --i-table ./data/filtered-feature-table.qza \
    --o-visualization ./data/filtered-feature-table_summary.qzv

Saved Visualization to: ./data/filtered-feature-table_summary.qzv


In [25]:
Visualization.load('./data/filtered-feature-table_summary.qzv')

<visualization: Visualization uuid: e6a080e5-3bd7-4571-9484-81ac91b3d5b1>

Alpha rarefaction

In [30]:
!qiime diversity alpha-rarefaction \
  --i-table ./data/filtered-feature-table.qza \
  --p-max-depth  10000 \
  --m-metadata-file ./data/fungut_metadata.tsv \
  --o-visualization ./data/alpha-rarefaction.qzv


Saved Visualization to: ./data/alpha-rarefaction.qzv


In [31]:
Visualization.load('./data/alpha-rarefaction.qzv')

<visualization: Visualization uuid: a74f5cc7-8694-4463-8eca-78e5acac8e1c>